<a href="https://colab.research.google.com/github/Manusua/03MIA_Algoritmos_de_Optimizacion/blob/main/Trabajo_Practico/Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos:  Manuel Suárez Román <br>
Url: [Trabajo Práctico](https://github.com/Manusua/03MIA_Algoritmos_de_Optimizacion/tree/main/Trabajo_Practico)<br>
Problema: Organizar los horarios de partidos de La Liga<br>


Descripción del problema:
Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia





                                        

In [342]:
import numpy as np
from tqdm import tqdm
import random
import copy
import os

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta
Para saber el número de posibilidades sin tener en cuentas las restricciones, debemos fijarnos en que podemos reducir el problema a asignar 10 partidos (ya definidos previamente) en 10 huecos, con la única condición de que puede haber más de un partido en un mismo hueco.

Se trata por lo tanto de Variaciones con Repetición con $n=10$ y $r=10$: $VR_{10}^{10} = 10^{10}$. En este caso $n$ representa el número de partidos de los que disponemos y $r$ el número de huecos.

Si nos fijamos nos encontramos con un caso en el que importa el orden (no es lo mismo un Celta - Real Madrid que un Valencia - Real Sociedad ya que, pese a que ambos son partidos de categoría A Vs categoría B, hemos tomado al consideración de que se tratan de elementos diferentes) y en el que existe repetición, pues en un mismo hueco puede haber más de un partido a la vez.
<br>

$$Posibilidades\;sin\;restricciones = 10^{10} = 10000000000.$$
<br>
Para el caso en el que existen restricciones, la única restricción de la que disponemos es que tanto el lunes como el viernes debe haber un partido (En el enunciado se indica que "Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes", por lo tanto debe ser uno, ni más ni menos), por lo que debemos hacer es calcular las diferentes posibilidades de asignar esos dos partidos en esos dos huecos y el problema que nos queda sería el mismo que el caso anterior con los 8 partidos restantes en 8 huecos, por lo que multiplicaríamos las combinaciones.

En este caso tenemos que calcular las formas de obtener 2 elementos de entre 10 sin repeticiones, además, nuevamente hemos interpretado que importa el orden, no es lo mismo que un partido se celebre el lunes a que se juegue el viernes (pese a que tengan ponderación de espectadores igual), se trata por lo tanto de una Variación sin Repetición con $n=10$ y $r=2$: $V_{10}^{2}=(10)\cdot(10-9) = 90$. Nuevamente $n$ represetna el número de partidos a repartir en $r$ huecos.

Por lo tanto existen 90 formas de encajar los partidos entre el Viernes y el Lunes. Resta el como cuadrar los otros 8 partidos en los 8 huecos restantes.
$VR_8^8 = 8^8$. Con todo esto, la solución final es:
<br>
$$Posibilidades\;con\;restricciones = 90\cdot 8^8 = 1509949440.$$

Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

De manera inicial, diseño el problema con una serie de clases. En concreto, clase equipo, partido (podria haber hecho directamente clase aprtido, pero así le aporto más abstracción al trabajo), clase horario y clase partido-horario, que es la clase resultante al combinar las dos anteriores.

In [350]:

MATRIZ_AUDIENCIAS = np.array([[2,1.3,1],[1.3,0.9,0.75],[1,0.75,0.47]])
VECTOR_COINCIDENCIAS = np.array([0,25,45,60,70,75,78,80,80])

class equipo:
  def __init__(self, nombre, abv, categoria):
    self.nombre = nombre
    self.abv = abv
    self.categoria = categoria
    if categoria == "A":
      self.index_categoria = 0
    elif categoria == "B":
      self.index_categoria = 1
    elif categoria == "C":
      self.index_categoria = 2
    else:
      # -1 para pues no deberiamos llegar aqui nunca
      self.index_categoria = -1
      print("Error, categroría de equipo no contemplada: " + str(nombre) + ', ' + str(categoria))

  def get_nombre(self):
    return self.nombre

  def get_abv(self):
    return self.abv

  def get_categoria(self):
    return self.categoria

class partido:
  def __init__(self, local, visitante):
    self.local = local
    self.visitante = visitante
    self.audiencia_base = MATRIZ_AUDIENCIAS[local.index_categoria][visitante.index_categoria]

  def __repr__(self):
    return str(self)

  def __str__(self):
    return self.get_abv()


  def get_abv(self):
    return self.local.get_abv() + ' - ' + self.visitante.get_abv()

  def get_nombre(self):
    return self.local.get_nombre() + ' - ' + self.visitante.get_nombre()

class horario:
  def __init__(self, nombre, factor_audiencia):
    self.nombre=nombre
    self.factor_audiencia = factor_audiencia

  def __repr__(self):
    return str(self)

  def __str__(self):
    return self.nombre + ', factor: ' + str(self.factor_audiencia)

  def __eq__(self, other):
    return self.nombre == other.nombre and self.factor_audiencia == other.factor_audiencia

class partido_horario:
  def __init__(self, partido, horario):
    self.nombre = horario.nombre +': ' + partido.get_nombre()
    self.abv = horario.nombre +': ' + partido.get_abv()
    self.horario = horario
    self.partido = partido
    self.audiencia_base = horario.factor_audiencia*partido.audiencia_base
    self.audiencia_coincidencias = horario.factor_audiencia*partido.audiencia_base

  def __repr__(self):
    return str(self)

  def __str__(self):
    return self.abv

  def get_nombre(self):
    return self.nombre

  def set_audiencia_coincidencias(self, audiencia_coincidencias):
    self.audiencia_coincidencias = audiencia_coincidencias

  def corregir_coincidencias(self, num_coincidencias):
    # Si hay dos partidos (dos coincidencias), accedemos al segundo elemento del array, indice 1
    num_coincidencias = int(num_coincidencias) - 1
    if num_coincidencias > 0 and num_coincidencias <=8:
      self.set_audiencia_coincidencias(self.audiencia_base * ((100-VECTOR_COINCIDENCIAS[num_coincidencias])/100))
    elif num_coincidencias < 0:
      print("Error, numero de coincidencias no puede ser negativo ni mayor que 8.")

In [286]:
# Inicializamos los datos
# Creamos todos los horarios
V20 = horario("V20", 0.4)
S12 = horario("S12", 0.55)
S16 = horario("S16", 0.7)
S18 = horario("S18", 0.8)
S20 = horario("S20", 1)
D12 = horario("D12", 0.45)
D16 = horario("D16", 0.75)
D18 = horario("D18", 0.85)
D20 = horario("D20", 1)
L20 = horario("L20", 0.4)
ARRAY_HORARIOS = [V20, S12, S16, S18, S20, D12, D16, D18, D20, L20]
#ARRAY_HORARIOS = [V20, S12, S20, D20, D16]

In [287]:
# Creamos todos los equipos
rma = equipo("Real Madrid", "rma", "A")
rso = equipo("Real Sociedad","rso", "A")
bcn = equipo("Barcelona", "bcn", "A")
clt = equipo("Celta", "cel", "B")
vlc = equipo("Valencia", "vlc", "B")
ath = equipo("Athletic", "ath", "B")
vll = equipo("Villarreal", "vll", "B")
ala = equipo("Alavés", "ala", "B")
lvt = equipo("Levante", "lev", "B")
esp = equipo("Espanyol", "esp", "B")
sev = equipo("Sevilla", "sev", "B")
bet = equipo("Betis", "bet", "B")
atl = equipo("Atlético", "atm", "B")
get = equipo("Getafe", "get", "B")
mal = equipo("Mallorca", "mal", "C")
eib = equipo("Eibar", "eib", "C")
leg = equipo("Leganés", "leg", "C")
osa = equipo("Osasuna", "osa", "C")
gra = equipo("Granada", "gra", "C")
val = equipo("Valladolid", "val", "C")

ARRAY_EQUIPOS = [rma, rso, bcn, clt, vlc, ath, vll, ala, lvt, esp, sev, bet, atl, get, mal, eib, leg, osa, gra, val]

In [288]:
# Creamos todos los partidos, pues los partidos no son aleatorioas, sino que son los dados en el enunciado
p1 = partido(clt, rma)
p2 = partido(vlc, rso)
p3 = partido(mal, eib)
p4 = partido(ath, bcn)
p5 = partido(leg, osa)
p6 = partido(vll, gra)
p7 = partido(ala, lvt)
p8 = partido(esp, sev)
p9 = partido(bet, val)
p10 = partido(atl, get)

ARRAY_PARTIDOS = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]
#ARRAY_PARTIDOS = [p1, p2, p3, p5, p10]

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

La función objetivo es

$Num\;audiencia = f(x) = \sum_{i=0}^7c_i \cdot b_i\cdot\sum_{j=1}^{n_i} p_{i,j},$
donde i es un índice que va desde el primer hueco (viernes a las 29h) hasta el último (lunes a las 20h), $n_i$ es el número de partidos que suceden a la vez en un determinado hueco $i$, $c_i$ es el factor corrector dependiendo del número de concurrencias en dicho horario y $b_i$ es la audiencia base de dicho horario y $p_{i,j}$ es la audiencia de cada uno de los partidos que se han decidido colocar en el horario $i$. Para analizar este valor se debe atender a la categoría del equipo local y del visitante, pero puesto que los partidos han sido fijados ya (solo hay que solucionar la jornada del enunciado), no es necesario formalizar esta parte.

En el sumatorio interno se ha decidido inicializar $j$ a 1 de forma que, en caso de que no haya ningún partido en dicho horario, $n_{i,j}=0$, por lo que tendríamos una suma vacía, 0.

Se trata de un problema de maximización, pues debemos hallar el máximo número de espectadores

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta
Un algoritmo de fuerza bruta es aquel que explore, una a una, todas las soluciones posibles y vaya almacenando la mejor de todas ellas. Es un método que garantiza hallar la solución exacta, pero en espacios de soluciones muy grandes (como en el que estamos) son impracticables.

In [289]:
VERBOSE = False

In [290]:
def get_array_coincidencias(arr_hor):
  arr = np.zeros(len(ARRAY_HORARIOS))
  for index, hor in enumerate(ARRAY_HORARIOS):
    arr[index] = arr_hor.count(hor)
  return arr

In [291]:
def correg_coincidencias(arr_part_hor):
  arr_hor = []
  for part_hor in arr_part_hor:
    arr_hor.append(part_hor.horario)
  arr_coinc = get_array_coincidencias(arr_hor)

  for elem in arr_part_hor:
    elem.corregir_coincidencias(arr_coinc[ARRAY_HORARIOS.index(elem.horario)])


In [292]:
def calc_audiencia(arr_part_hor):
  correg_coincidencias(arr_part_hor)
  aud_tot = 0
  for elem in arr_part_hor:
    aud_tot = aud_tot + elem.audiencia_coincidencias
  return aud_tot

In [293]:
# Una vez  que sabemos cuantos partidos hay en cada horario, iteramos sobre todas las posibilidades de asignación de partidos en ese horario

# Todas las audiencias suman, por lo que no debemos preocuparnos por casos en los que haya partidos no asignados, pues esa solución nunca será marcada como la mejor
def asignar_partidos(arr_hor, arr_part, arr_part_hor):
  # Encontramos el primer horario en el que debe haber un partido
  arr_nonz = np.nonzero(arr_hor)[0]

  if len(arr_nonz) > 0:
    index_hor = arr_nonz[0]
    # Asignamos en ese horario un partido (cubrimos todas las posibilidades)
    for part in arr_part:
      arr_hor_aux = arr_hor.copy()
      arr_part_hor_aux = arr_part_hor.copy()
      arr_part_aux = arr_part.copy()

      arr_part_aux.remove(part)
      arr_part_hor_aux.append(partido_horario(part, ARRAY_HORARIOS[index_hor]))
      arr_hor_aux[index_hor] -= 1
      audiencia, arr_part_hor_sol = asignar_partidos(arr_hor_aux, arr_part_aux, arr_part_hor_aux)
  else:
    # Ya hemos asignado todos los partidos
    audiencia = calc_audiencia(arr_part_hor)
    arr_part_hor_sol = arr_part_hor
  return audiencia, arr_part_hor_sol

In [294]:
def asignar_horarios(arr_hors, partidos_restantes, index):
  arr_aux = arr_hors.copy()
  max_aux = 0
  arr_max_hors = []
  part_rest = partidos_restantes
  for i in range(partidos_restantes + 1):
    part_rest = partidos_restantes
    if index == len(arr_aux)-1:
      #Asignamos todos los partidos al último índice porque es la última opción
      arr_aux[index] = part_rest
      part_rest = 0
    else:
      arr_aux[index] = i
      part_rest -= i
    if part_rest == 0:
      arr_part_hor = []
      aud, arr_sols = asignar_partidos(arr_aux, ARRAY_PARTIDOS, arr_part_hor)
    else:
      aud, arr_sols = asignar_horarios(arr_aux, part_rest, index + 1)
    if aud > max_aux:
      max_aux = aud
      arr_max_hors = arr_sols
  return max_aux, arr_max_hors

In [ ]:
num_partidos = len(ARRAY_PARTIDOS)
num_horarios = len(ARRAY_HORARIOS)

# En num_partidos_horario almacenamos el número de partidos en cada horario, inicialmente 0 y vamos a iterar sobre todas las soluciones
num_partidos_horario = np.zeros(num_horarios)
audiencia, arr_sols = asignar_horarios(num_partidos_horario, num_partidos, 0)
print('SOLUCIONES:')
print("Audiencia total: " + str(audiencia))
print("Partidos: " + str(arr_sols))


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta
Tenemos que analizar el número de bucles anidados que tenemos:
  * ```for i in range(partidos_restantes + 1):```
  Este bucle se repite reduciendo en uno el numer de partidos restantes, por lo que, al finalizar la recursividad tendríamos $n!$, siendo $n$ el número de partidos.
  *  ```for part in arr_part::``` Nuevamente se trata de un bucle que induce una recursividad en la que va descendiendo en una unidad el tamaño de ```arr_part``` a cada paso, por lo que tenemos $n!$ nuevamente.
  * Dentro de la función ```calcular_asistencia``` tenemos tres bucles que iteran sobre el arraay que agrupa partidos y horarios, por lo tanto $3n$.


Teniendo en cuenta el número de bucles anidados que estamos tratando, la complejidad del algoritmo es de tipo $O(f) = (n!)^2*3n$, siendo $n$ el número de partidos. Es decir, se tata de una complejidad horrible que crece mucho con cada nuevo elemento que añadimos.



(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

Voy a usar un algoritmo genético que mejore la exploración de soluciones

Al realizar una poda de opciones, estamos descartando evaluar soluciones que sabemos que no van a darnos una solución buena, por lo que la complejidad del algorimto diseñado será mejor

In [364]:
# Para printear ordenada la solución
def indice_horario(ph):
    return ["V20", "S12", "S16", "S18", "S20", "D12", "D16", "D18", "D20", "L20"].index(ph.horario.nombre)

In [317]:
def calc_max_aud_poblacion(poblacion):
  aud_max = 0
  part_hor_max = []
  for indiv in poblacion:
    aud_ind = calc_audiencia(indiv)
    if aud_ind > aud_max:
      aud_max = aud_ind
      part_hor_max = indiv
  return aud_max, part_hor_max


In [318]:
def assign_horarios(arr_p, arr_h):
  arr_part_hor = []
  part_l, part_v = random.sample(arr_p, 2)

  # Asignacion partido lunes
  arr_part_hor.append(partido_horario(part_l, arr_h[0]))
  # Asignación partido viernes
  arr_part_hor.append(partido_horario(part_v, arr_h[-1]))
  # Eliminamos de elegibles lunes y viernes y los partidos seleccionados
  arr_p.remove(part_l)
  arr_p.remove(part_v)
  arr_h.pop(0)
  arr_h.pop(-1)
  # Asignamos el resto de partidos de forma aleatoria sin restricciones
  while(len(arr_p) > 0):
    part = random.sample(arr_p, 1)[0]
    hor = random.sample(arr_h, 1)[0]
    arr_part_hor.append(partido_horario(part, hor))
    arr_p.remove(part)
  return arr_part_hor

In [319]:
def crear_pob_inicial(tam_poblacion, array_partidos, array_horarios):
  poblacion = []
  for _ in range(tam_poblacion):
    # Seleccionamos aleatoriamente el orden de los partidos y su ubicación garantizando que hay uno el viernes y uno el lunes
    # Para ello los asignamos inicialmente y los eliminamos de los horarios candidatos
    arr_p_aux = array_partidos.copy()
    arr_h_aux = array_horarios.copy()
    poblacion.append(assign_horarios(arr_p_aux,arr_h_aux))
  return poblacion

In [320]:
def generate_mask(len):
    return [random.choice([0, 1]) for _ in range(len)]

In [321]:
# Se intercambian dos partidos de horario de forma aleatoria
def mutate(individuo, prob_mutacion):
  if random.random() < prob_mutacion:
    index1, index2 = sorted(random.sample(range(len(individuo)), 2))
    part_hor1 = individuo[index1]
    part_hor2 = individuo[index2]
    part1_new = partido_horario(part_hor1.partido, part_hor2.horario)
    part2_new = partido_horario(part_hor2.partido, part_hor1.horario)
    ind_aux = individuo[:index1] + [part1_new] + individuo[index1+1:index2] + [part2_new] + individuo[index2+1:]
    return ind_aux
  else:
    return individuo

In [322]:
# Opción para cruce uniforme con una máscara aleatoria para cada descendencia
def cruce_pob_mask_random(poblacion, prob_mutacion):

  poblacion_copia = copy.deepcopy(poblacion)
  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1:
    p1, p2 = random.sample(poblacion_copia, 2)
    poblacion_copia.remove(p1)
    poblacion_copia.remove(p2)
    mask = generate_mask(len(p1))
    descendencia = generate_desc([p1, p2], prob_mutacion, mask)
    poblacion_final.extend(descendencia)
  return poblacion_final

In [323]:
# No se pueden jugar un partido dos veces, así que hay que tener un mecanismo de control para esto
# Ojo, en el mecanismo de controlo debemso tener en cuenta que, para ser solución viable, lunes y viernes debe haber un partido
#
# Me he tenido que crear unos arrays complementarios con las strings de los aprtidos, porque si no el operador "in" no funcionaba
# correctamente cuando había que hallar los partidos desaparecidos
def factibilice(individuo, arr_partidos_hor_init):
  arr_partidos = []
  arr_partidos_st = []
  arr_partidos.extend(part_hor.partido for part_hor in arr_partidos_hor_init)
  arr_partidos_st.extend(part_hor.partido.get_nombre() for part_hor in arr_partidos_hor_init)
  part_individuo = []
  part_individuo_st = []
  part_individuo.extend(part_hor.partido for part_hor in individuo)
  part_individuo_st.extend(part_hor.partido.get_nombre() for part_hor in individuo)

  # Añadimos el random.shuffle para aleatorizar también la factibilización
  partidos_desaparecidos = []
  for index, part in enumerate(arr_partidos):
    if arr_partidos_st[index] not in part_individuo_st:
      partidos_desaparecidos.append(part)
  random.shuffle(partidos_desaparecidos)

  part_hor_aux = []

  # En todos los casos, los horarios generados en part_hor_aux vienen del padre, que es factible, por lo que, pese a que no aleatorizamos del todo,
  # se garantiza que lunes y viernes va a haber un partido (los había en el padre)

  for i in range(len(part_individuo)):

    if part_individuo_st[i] in part_individuo_st[:i]:
      p = partidos_desaparecidos.pop(0)
      h = arr_partidos_hor_init[i].horario
      p_h = partido_horario(p, h)
      #print("Factibilizando (" + str(i) + ") " + str(part_individuo[i]) + ". Añadiremos " + str(p_h))
      part_hor_aux.append(p_h)
    else:
      part_hor_aux.append(individuo[i])
  return part_hor_aux

In [324]:
def generate_desc(arr_padres, prob_mutacion, mask):

  # Si el elemento de la máscara es 1, entonces se elige el segundo padre, sino el primero
  hijo1 = []
  hijo2 = []
  for index, bit in enumerate(mask):

    hijo1.append(arr_padres[bit][index])
    hijo2.append(arr_padres[not bit][index])

  # En cualquiera de los dos padres etá una lista de partidos con todos los partidos,
  # pues sabemos que los padres son factibles
  hijo1 =  factibilice(hijo1, arr_padres[0])
  hijo2 =  factibilice(hijo2, arr_padres[0])

  mut1 = mutate(hijo1, prob_mutacion)
  mut2 = mutate(hijo2, prob_mutacion)

  return [hijo1, hijo2, mut1, mut2]

In [325]:
# Vamos a usar un mecanismo de cruce uniforme con una máscara igual para cada descendencia
def cruce_pob_mask_unif(poblacion, prob_mutacion, mask):
  poblacion_copia = copy.deepcopy(poblacion)
  poblacion_final = copy.deepcopy(poblacion)

  while len(poblacion_copia) > 1:
    p1, p2 = random.sample(poblacion_copia, 2)
    poblacion_copia.remove(p1)
    poblacion_copia.remove(p2)
    descendencia = generate_desc([p1, p2], prob_mutacion, mask)
    poblacion_final.extend(descendencia)
  return

In [326]:
def selecc_pob(poblacion, tam_poblacion, prob_elitismo):
  poblacion_ordenada = sorted([[calc_audiencia(solucion), solucion] for solucion in poblacion ], key= lambda x:x[0], reverse=True)

  #Devolvemos porcentaje determinado por elitismo de mejores soluciones y el resto se eligen aleatoriamente
  return [x[1] for x in poblacion_ordenada][:int(tam_poblacion*prob_elitismo)] + random.sample([x[1] for x in poblacion_ordenada][int(tam_poblacion*prob_elitismo):], int(tam_poblacion*(1-prob_elitismo)))

In [359]:
def alg_genetico(array_partidos, array_horarios, tam_poblacion, prob_mutacion, prob_elitismo, generaciones):

  # Creamos una población inicial
  poblacion = crear_pob_inicial(tam_poblacion, array_partidos, array_horarios)
  # Realizamos una evaluación inicial
  aud_max, part_hor_max = calc_max_aud_poblacion(poblacion)
  sol = (aud_max, part_hor_max)

  # Ejecutamos el algoritmo genético tantas veces como generaciones se hayan indicado
  cond_parada = 0
  pbar = tqdm(total=generaciones)
  mask = generate_mask(len(array_partidos))
  while(cond_parada < generaciones):
    #Cruce de la poblacion(incluye mutación)
    #poblacion = cruce_pob_mask_unif(poblacion, prob_mutacion, mask)
    poblacion = cruce_pob_mask_random(poblacion, prob_mutacion)
    #Seleccionamos la población en funcion del elitismo
    poblacion = selecc_pob(poblacion, tam_poblacion, prob_elitismo)
    #Evaluamos la nueva población
    aud_max, part_hor_max = calc_max_aud_poblacion(poblacion)
    sol = (aud_max, part_hor_max)

    print("Generacion #", str(cond_parada), ": audiencia " , str(round(sol[0], 5)) , ", solución es:" , str(sol[1]))
    pbar.update(1)
    cond_parada += 1
  return sol

In [366]:
# Determinamos los parámetros para el algoritmo genético
array_partidos, array_horarios = ARRAY_PARTIDOS, ARRAY_HORARIOS
tam_poblacion=500
prob_mutacion=0.3
prob_elitismo=0.4
generaciones=150

In [365]:
sol = alg_genetico(array_partidos, array_horarios, tam_poblacion, prob_mutacion, prob_elitismo, generaciones)

print("-------------------------------------\nLa solución encontrada ha sido: ")
for ph in sorted(sol[1], key=indice_horario):
  print(ph.get_nombre())
print("Con una audiencia total de ", str(round(sol[0], 5)), " millones de espectadores.")


 35%|███▌      | 7/20 [06:01<11:11, 51.63s/it]

  1%|          | 1/150 [00:00<01:21,  1.84it/s]

Generacion # 0 : audiencia  6.608 , solución es: [V20: mal - eib, L20: bet - val, S18: esp - sev, S16: vlc - rso, D16: leg - osa, S18: atm - get, S12: vll - gra, D18: ala - lev, S20: ath - bcn, D20: cel - rma]



  1%|▏         | 2/150 [00:00<00:59,  2.47it/s]

Generacion # 1 : audiencia  6.608 , solución es: [V20: mal - eib, L20: bet - val, S18: esp - sev, S16: vlc - rso, D16: leg - osa, S18: atm - get, S12: vll - gra, D18: ala - lev, S20: ath - bcn, D20: cel - rma]



  2%|▏         | 3/150 [00:01<00:53,  2.76it/s]

...

Generacion # 147 : audiencia  6.856 , solución es: [V20: mal - eib, L20: leg - osa, D12: bet - val, S12: vll - gra, D20: vlc - rso, S18: atm - get, S20: cel - rma, S16: esp - sev, D18: ath - bcn, D16: ala - lev]



 99%|█████████▉| 149/150 [02:27<00:00,  1.05it/s]

Generacion # 148 : audiencia  6.856 , solución es: [V20: mal - eib, L20: leg - osa, D12: bet - val, S12: vll - gra, D20: vlc - rso, S18: atm - get, S20: cel - rma, S16: esp - sev, D18: ath - bcn, D16: ala - lev]



100%|██████████| 150/150 [02:29<00:00,  1.01it/s]

Generacion # 149 : audiencia  6.856 , solución es: [V20: mal - eib, L20: leg - osa, D12: bet - val, S12: vll - gra, D20: vlc - rso, S18: atm - get, S20: cel - rma, S16: esp - sev, D18: ath - bcn, D16: ala - lev]
-------------------------------------
La solución encontrada ha sido: 
V20: Mallorca - Eibar
S12: Villarreal - Granada
S16: Espanyol - Sevilla
S18: Atlético - Getafe
S20: Celta - Real Madrid
D12: Betis - Valladolid
D16: Alavés - Levante
D18: Athletic - Barcelona
D20: Valencia - Real Sociedad
L20: Leganés - Osasuna
Con una audiencia total de  6.856  millones de espectadores.


(*)Calcula la complejidad del algoritmo

Respuesta
LA comlejidad del algoritmo viene determinada principalmente por el numero de generaciones y por el tamaño de la población, siendo el algoritmo de tipo $O(f) = N\cdot g$, siendo $N$ el tamaño de la población y $g$ el número de generaciones sobre las que se itera. Además de esto, hay una serie de operaciones que, si quisieramos hacer un análisis mucho más detallado, estarían sumando a $N$ derivadas de cruzar y seleccionar a la población, de forma que quedaría $(N+c)$. No obstante, como estamos usando notación de Landau, podemos resumir que el orden de complejidad del algoritmo es $O(f) = N\cdot g$.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

Vamos a crear un numero aletorio de partidos entre 0 y 10 en el que se van a enfrentar equipos aleatorios y vamos a seleccionar una serie de horarios en la que deben colocarse dichos partidos.

In [371]:
num_part = random.randint(1, 10)
num_hor = random.randint(1, 10)

arr_equipos_aux = ARRAY_EQUIPOS.copy()
arr_horarios_aux = ARRAY_HORARIOS.copy()
arr_horarios_generados = []
arr_partidos_generados = []

for i in range(num_part):
  index_local = random.randint(0,len(arr_equipos_aux)-1)
  local = arr_equipos_aux[index_local]
  arr_equipos_aux.remove(local)

  index_visitante = random.randint(0,len(arr_equipos_aux)-1)
  visitante = arr_equipos_aux[index_visitante]
  arr_equipos_aux.remove(visitante)

  arr_partidos_generados.append(partido(local, visitante))

for i in range(num_hor):
  index_horario = random.randint(0,len(arr_horarios_aux)-1)
  horario = arr_horarios_aux[index_horario]
  arr_horarios_aux.remove(horario)

  arr_horarios_generados.append(horario)

print("Partidos generados aleatoriamente: " + str(arr_partidos_generados))
print("Horarios generados aleatoriamente: " + str(arr_horarios_generados))

Partidos generados aleatoriamente: [val - osa, ala - rma, ath - rso, lev - gra, bcn - vlc, leg - bet, mal - eib, cel - atm, esp - vll]
Horarios generados aleatoriamente: [S12, factor: 0.55, D12, factor: 0.45, V20, factor: 0.4]


Aplica el algoritmo al juego de datos generado

Respuesta
Por la forma de construir la solución es muy simple, únicamente es necesario sobreescribir los arrays de partidos y horarios: ```ARRAY_PARTIDOS, ARRAY_HORARIOS```, por lo que ahora pasaremos el array de partidos y de horarios generados.

In [372]:
# Determinamos los parámetros para el algoritmo genético
array_partidos, array_horarios = arr_partidos_generados, arr_horarios_generados
tam_poblacion=500
prob_mutacion=0.3
prob_elitismo=0.4
generaciones=150

In [373]:
sol = alg_genetico(array_partidos, array_horarios, tam_poblacion, prob_mutacion, prob_elitismo, generaciones)
print("-------------------------------------\nLa solución encontrada ha sido: ")
for ph in sorted(sol[1], key=indice_horario):
  print(ph.get_nombre())
print("Con una audiencia total de ", str(round(sol[0], 5)), " millones de espectadores.")

  1%|          | 1/150 [00:00<01:05,  2.29it/s]

Generacion # 0 : audiencia  1.78346 , solución es: [S12: ath - rso, D12: val - osa, D12: leg - bet, D12: mal - eib, D12: lev - gra, D12: esp - vll, V20: ala - rma, D12: bcn - vlc, D12: cel - atm]


  1%|▏         | 2/150 [00:00<00:48,  3.06it/s]

Generacion # 1 : audiencia  1.897 , solución es: [S12: bcn - vlc, V20: cel - atm, D12: val - osa, D12: mal - eib, D12: esp - vll, D12: lev - gra, V20: ala - rma, D12: leg - bet, D12: ath - rso]


  2%|▏         | 3/150 [00:01<00:52,  2.81it/s]

...

Generacion # 147 : audiencia  2.23015 , solución es: [S12: ath - rso, V20: val - osa, V20: leg - bet, V20: mal - eib, D12: cel - atm, D12: esp - vll, D12: ala - rma, V20: lev - gra, S12: bcn - vlc]


 99%|█████████▉| 149/150 [01:49<00:00,  1.31it/s]

Generacion # 148 : audiencia  2.23015 , solución es: [S12: ath - rso, V20: val - osa, V20: leg - bet, V20: mal - eib, D12: cel - atm, D12: esp - vll, D12: ala - rma, V20: lev - gra, S12: bcn - vlc]


100%|██████████| 150/150 [01:50<00:00,  1.36it/s]

Generacion # 149 : audiencia  2.23015 , solución es: [S12: ath - rso, V20: val - osa, V20: leg - bet, V20: mal - eib, D12: cel - atm, D12: esp - vll, D12: ala - rma, V20: lev - gra, S12: bcn - vlc]
-------------------------------------
La solución encontrada ha sido: 
V20: Valladolid - Osasuna
V20: Leganés - Betis
V20: Mallorca - Eibar
V20: Levante - Granada
S12: Athletic - Real Sociedad
S12: Barcelona - Valencia
D12: Celta - Atlético
D12: Espanyol - Villarreal
D12: Alavés - Real Madrid
Con una audiencia total de  2.23015  millones de espectadores.


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

* Manual Master IA VIU
* Notebook clase Algoritmo Genético para TSP
* [Introducción a los Algoritmos Genéticos](http://sabia.tic.udc.es/mgestal/cv/AAGGtutorial/). Marcos Gestal Pose. Universidad de Coruña
* [Tema 2. Algoritmos Genéticos](http://www.sc.ehu.es/ccwbayes/docencia/mmcc/docs/t2geneticos). Abdelmalik Moujahid, Iñaki Inza y Pedro Larrañaga. Departamento de Ciencias de la Computación e Inteligencia Artificial. Universidad del País Vasco–Euskal Herriko Unibertsitatea

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta

La primera de las variaciones (de hecho la estructura de clases está pensada en este sentido) es aleatorizar los partidos. De esta forma podemos transformar el ejercicio en una búsqueda de los partidos y horarios óptimos para maximizar audiencia. A nivel de programación, simplemente consistiría en aleatorizar el array ```ARRAY_PARTIDOS```.

Otra complicación podría ser que en al audiencia influyera qué equipo es local y qué equipo visitante. Se trata de hecho, de un comportamiento habitual en LaLiga, pues hay equipo que tienen fama de ser más díficiles de ganar cuando juegan en casa, lo que puede añadir atractivo al partido. Esto quedaría solventado añadiendo un factor de corrección al emplearse la función ```corregir_coincidencias()```. En caso de una serie de datos más complicados para determinar este caso, estos podrían almacenarse en una matriz y hacer una búsqueda de índices de equipos por filas y columnas en caso de que estos sean locales y/o visitantes.

Otra complicación sería añadir más restricciones. Estas podrían modelar casos de la vida real en caso de que ciertos horarios de una determinada jornada coincidiera con otro tipo de eventos a nivel nacional y/o internacional. En este caso habría que hacer un estudio de restricciones un poco más detallado en al construcción de los individuos de la población, garantizando que estos siempre cumplan dichas restricciones.